In [1]:
# ==============================
# 1. Load Dataset
# ==============================
import pandas as pd

file_path = "/Users/kirancorreya/Downloads/Dessertation/fies/Dataset2025.xlsx"
df = pd.read_excel(file_path)

# Add Job ID as row number
df.insert(0, 'Job ID', range(1, len(df) + 1))


print("Dataset loaded. Shape:", df.shape)
print("Columns:", df.columns)

Dataset loaded. Shape: (6330, 10)
Columns: Index(['Job ID', 'Job Title', 'Company', 'Location', 'Minimum Salary',
       'Maximum Salary', 'Date Posted', 'Applications Count', 'Search Keyword',
       'Full Job Description'],
      dtype='object')


In [2]:

# ==============================
# 2. Match Keywords in Job Titles
# ==============================
import re

keywords = ['data', 'data analyst', 'power', 'AI', 'data engineer', 'data scientist', 'data science',
            'Sales Analyst', 'Business Analyst', 'Finance Analyst', 'Research Analyst', 'machine learning',
            'python', 'Intelligence']

def match_keywords_in_title(job_title):
    matched = [kw for kw in keywords if re.search(r'\b' + re.escape(kw.lower()) + r'\b', str(job_title).lower())]
    return ', '.join(matched) if matched else 'No Keyword'

if 'Matched Keywords' not in df.columns:
    df['Matched Keywords'] = df['Job Title'].apply(match_keywords_in_title)

print("Matched Keywords column added.")

# NEW: Count how many rows have 'No Keyword'
keyword_count = (df['Matched Keywords'] != 'No Keyword').sum()
print(f"Number of job titles with matched keyword: {keyword_count}")



Matched Keywords column added.
Number of job titles with matched keyword: 1561


In [7]:

# ==============================
# 3. Extract Skills from Job Descriptions
# ==============================
skills = ['python', 'sql', 'r', 'machine learning',
          'tableau', 'snowflake', 'aws', 'excel',
          'power bi']

short_skills = ['r']
long_skills = [skill for skill in skills if skill not in short_skills]

def extract_skills(description):
    description_cleaned = re.sub(r'[^a-zA-Z]', '', str(description).lower())
    matched_skills = []

    for skill in long_skills:
        if skill.replace(" ", "").lower() in description_cleaned:
            matched_skills.append(skill)

    for short_skill in short_skills:
        if re.search(r'\b' + re.escape(short_skill) + r'\b', str(description), re.IGNORECASE):
            matched_skills.append(short_skill)

    return matched_skills if matched_skills else "No skills found"

df['Matched Skills'] = df['Full Job Description'].apply(extract_skills)
print("Skills extracted.")



Skills extracted.


In [8]:

# ==============================
# 4. Salary Standardization
# ==============================
def detect_salary_type(salary):
    if pd.isna(salary):
        return None
    if 0 <= salary <= 96:
        return 'Hourly'
    elif 97 <= salary <= 479:
        return 'Daily'
    elif 480 <= salary <= 2099:
        return 'Weekly'
    elif 2100 <= salary <= 6500:
        return 'Monthly'
    else:
        return 'Yearly'

def get_salary_type(row):
    if pd.notna(row['Minimum Salary']):
        return detect_salary_type(row['Minimum Salary'])
    elif pd.notna(row['Maximum Salary']):
        return detect_salary_type(row['Maximum Salary'])
    return None

df['Salary Type'] = df.apply(get_salary_type, axis=1)

def convert_to_annual(row):
    min_salary = row['Minimum Salary']
    max_salary = row['Maximum Salary']
    avg_salary = None

    if pd.notna(min_salary) and pd.notna(max_salary):
        avg_salary = (min_salary + max_salary) / 2
    elif pd.notna(min_salary):
        avg_salary = min_salary
    elif pd.notna(max_salary):
        avg_salary = max_salary
    else:
        return None

    salary_type = row['Salary Type']

    if salary_type == 'Hourly':
        return avg_salary * 40 * 52
    elif salary_type == 'Daily':
        return avg_salary * 5 * 52
    elif salary_type == 'Weekly':
        return avg_salary * 52
    elif salary_type == 'Monthly':
        return avg_salary * 12
    elif salary_type == 'Yearly':
        return avg_salary
    return None

df['Standardised Annual Salary'] = df.apply(convert_to_annual, axis=1)

print("Salaries standardized.")



Salaries standardized.


In [9]:

# ==============================
# 5. Save Processed Dataset
# ==============================
output_path = "/Users/kirancorreya/Downloads/Dessertation/fies/Final_Processed_Dataset2025.xlsx"
df.to_excel(output_path, index=False)

print("Final dataset saved at:", output_path)



Final dataset saved at: /Users/kirancorreya/Downloads/Dessertation/fies/Final_Processed_Dataset2025.xlsx
